In [21]:
import logging
import threading
import speech_recognition as sr
import openai
import os
from pathlib import Path
import pygame
import tempfile

from dotenv import load_dotenv

load_dotenv()



True

In [27]:


class VoiceAssistant:
    def __init__(self, api_key: str):
        """Initialize the voice assistant with required components."""
        # Set up logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

        # OpenAI setup
        openai.api_key = api_key

        # Initialize speech recognizer
        self.recognizer = sr.Recognizer()
        self.is_speaking = threading.Event()
        self.interrupted = threading.Event()
        self.speech_lock = threading.Lock()

        # Initialize pygame mixer for audio playback
        pygame.mixer.init()

    def speak_text(self, text: str):
        """Convert text to speech using OpenAI's TTS and play it."""
        self.logger.info(f"Speaking: {text}")
        self.is_speaking.set()
        self.interrupted.clear()

        try:
            # Create a temporary file to store the speech audio
            with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as temp_file:
                # Generate speech using OpenAI's TTS
                response = openai.audio.speech.create(
                    model="tts-1",
                    voice="alloy",  # Options: alloy, echo, fable, onyx, nova, shimmer
                    input=text
                )
                
                # Save the audio to the temporary file
                response.stream_to_file(temp_file.name)
                
                # Play the audio
                pygame.mixer.music.load(temp_file.name)
                pygame.mixer.music.play()
                
                # Wait for the audio to finish playing
                while pygame.mixer.music.get_busy() and not self.interrupted.is_set():
                    pygame.time.Clock().tick(10)
                
                # Clean up
                pygame.mixer.music.stop()
                os.unlink(temp_file.name)
                
        except Exception as e:
            self.logger.error(f"Error in text-to-speech: {e}")
        finally:
            self.is_speaking.clear()

    def listen_for_speech(self) -> str:
        """Listen for and convert speech to text."""
        with sr.Microphone() as source:
            self.logger.info("Listening for your question...")
            self.recognizer.adjust_for_ambient_noise(source, duration=1)
            try:
                audio_data = self.recognizer.listen(source, timeout=10, phrase_time_limit=30)
                text = self.recognizer.recognize_google(audio_data, language='en-US')
                self.logger.info(f"Recognized text: {text}")
                return text
            except sr.WaitTimeoutError:
                return "No speech detected."
            except sr.UnknownValueError:
                return "Could not understand the audio."
            except Exception as e:
                self.logger.error(f"Error in speech recognition: {e}")
                return f"Error during speech recognition: {str(e)}"

    def get_gpt_response(self, prompt: str) -> str:
        """Get response from GPT model."""
        try:
            self.logger.info(f"Sending prompt to GPT: {prompt}")
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that provides clear, concise answers."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=150
            )
            response_text = response.choices[0].message.content.strip()
            self.logger.info(f"Received GPT response: {response_text}")
            return response_text
        except Exception as e:
            self.logger.error(f"Error getting GPT response: {e}")
            return "I apologize, but I encountered an error processing your request."

    def run(self):
        """Main loop for running the assistant."""
        self.logger.info("Starting voice assistant...")
        self.speak_text("Voice assistant is ready. You can start speaking.")

        try:
            while True:
                # Listen for user input
                user_input = self.listen_for_speech()

                # Process valid input
                if user_input.lower() in ["exit", "quit", "goodbye"]:
                    self.speak_text("Goodbye!")
                    self.logger.info("Exiting assistant...")
                    break

                if user_input and user_input not in ["No speech detected.", "Could not understand the audio."]:
                    self.logger.info(f"Processing user input: {user_input}")

                    # Get GPT response
                    response = self.get_gpt_response(user_input)
                    self.logger.info(f"Speaking response: {response}")

                    # Speak the response
                    self.speak_text(response)

        except KeyboardInterrupt:
            self.logger.info("Assistant terminated by user")
            self.speak_text("Shutting down")


In [28]:
import os
from dotenv import load_dotenv
import openai


def main():
    # Load environment variables
    load_dotenv()

    # Fetch the OpenAI API key from the environment
    api_key = os.getenv("OPEN_AI_API_KEY_VOICE")

    # Check if the API key is present
    if not api_key:
        print("Error: OPENAI_API_KEY_VOICE not found in environment variables.")
        return

    try:
        # Initialize and run the assistant
        assistant = VoiceAssistant(api_key)
        assistant.run()
    except Exception as e:
        print(f"Error running assistant: {e}")

if __name__ == "__main__":
    main()


INFO:__main__:Starting voice assistant...
INFO:__main__:Speaking: Voice assistant is ready. You can start speaking.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"
/tmp/ipykernel_38864/1543711622.py:37: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(temp_file.name)
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init n